## Rules:
* Men
    * Team must contain exactly 25 riders
    * Budget is 150 points
    * Team must contain not more than 1 rider costing 24 points or more
    * Team must contain not more than 3 riders costing 18 points or more
* Women
    * Team must contain exactly 15 riders
    * Budget is 150 points
    * May spend not more than 100 **points** on riders costing 20 points or more
    
    
    
Let's express all our constraints as a tuple/list.

* Men
    * Points remaining to spend
    * 24+ point slots remaining
    * 18+ point slots remaining
    * Rider slots remaining
        * e.g. (83, 0, 1, 15)
    * So, to the Men records, add some columns:
        * 24+
        * 18+
* Women
    * Points remaining to spend
    * Points remaining to spend on 20+ riders
    * Rider slots remaining
        * e.g. (83, 60, 8)
    * So, to the Women records, add a column:
        * 20+ (maybe??)

In [1]:
import pandas as pd
import time
import random

In [2]:
# Create all_riders DataFrame
men = pd.read_csv('../data/FSA_DS_Men2019.csv')
men = men[['Rider Name', 'Price', 'Score 2019']]
men['24+'] = (men['Price'] >= 24).astype(int)
men['18+'] = (men['Price'] >= 18).astype(int)
men

,Rider Name,Price,Score 2019,24+,18+
0,Primož Roglič,28,4070,1,1
1,Alejandro Valverde,34,2910,1,1
2,Egan Bernal,24,2752,1,1
3,Julian Alaphilippe,30,2601,1,1
4,Pascal Ackermann,20,2462,0,1
...,...,...,...,...,...
1373,Edoardo Zardini,1,0,0,0
1374,Eugert Zhupa,1,0,0,0
1375,Kamil Zielinski,1,0,0,0
1376,Maikel Zijlaard,1,0,0,0


In [3]:
# Keep only riders who scored at least 50 points
scorers = men[men['Score 2019'] >= 50]
len(scorers)

477

In [6]:
for n in range(25, 101, 5):
    subsample = tuple(random.sample(list(scorers.index), n))
    print('Now solving random sample size {}...'.format(n))
    t0 = time.perf_counter()
    val, taken = knapsack_men(subsample, (150, 1, 3, 25))
    t1 = time.perf_counter()
    print('Hired {} riders costing total {} points, got total score of {}'.format(len(taken),
                                  men[men.index.isin(taken)]['Price'].sum(), val))
    print('{:.2f} seconds elapsed'.format(t1-t0))
    print()

Now solving random sample size 25...
Hired 23 riders costing total 114 points, got total score of 10601
6.27 seconds elapsed

Now solving random sample size 30...
Hired 25 riders costing total 97 points, got total score of 9074
4.71 seconds elapsed

Now solving random sample size 35...
Hired 25 riders costing total 85 points, got total score of 7898
6.60 seconds elapsed

Now solving random sample size 40...
Hired 25 riders costing total 147 points, got total score of 12831
39.77 seconds elapsed

Now solving random sample size 45...
Hired 25 riders costing total 149 points, got total score of 20979
30.82 seconds elapsed

Now solving random sample size 50...
Hired 25 riders costing total 146 points, got total score of 17101
44.95 seconds elapsed

Now solving random sample size 55...
Hired 25 riders costing total 116 points, got total score of 12901
47.18 seconds elapsed

Now solving random sample size 60...
Hired 25 riders costing total 148 points, got total score of 21563
75.66 seconds 

In [3]:
# Create a mini DataFrame for fast(er) testing
men_mini = men[(men['Score 2019'] > 0) & (men.index % 10 == 0)]
men_mini

,Rider Name,Price,Score 2019,24+,18+
0,Primož Roglič,28,4070,1,1
10,Mathieu van der Poel,8,1778,0,0
20,Mike Teunissen,2,1507,0,0
30,David Gaudu,2,1285,0,0
40,Jasper Stuyven,26,1100,1,1
...,...,...,...,...,...
600,Aksel Nommela,1,10,0,0
610,Orluis Aular,1,10,0,0
620,August Jensen,1,8,0,0
630,Brenton Jones,1,3,0,0


In [4]:
def knapsack_men(to_consider, avail, memo={}):
    # to_consider is a tuple of rider indices
    # avail is a tuple of (points remaining to spend, 24+ point slots remaining, 18+ point slots remaining, all slots remaining)
    # we're also going to query a global 'men' DataFrame variable, bite me
    
    if (to_consider, avail) in memo:
        result = memo[(to_consider, avail)]
    elif to_consider == () or avail[0] == 0 or avail[3] == 0:
        result = (0, ())
    else:
        next_rider = men.loc[to_consider[0]]
        if (next_rider['Price'] > avail[0]) or (next_rider['24+'] > avail[1]) or (next_rider['18+'] > avail[2]):
            #Explore right branch only
            result = knapsack_men(to_consider[1:], avail, memo)
        else:
            #Explore left branch
            with_avail = (avail[0] - next_rider['Price'],
                          avail[1] - next_rider['24+'],
                          avail[2] - next_rider['18+'],
                          avail[3] - 1)
            with_val, with_team = knapsack_men(to_consider[1:], with_avail, memo)
            with_val += next_rider['Score 2019']
            #Explore right branch
            without_val, without_team = knapsack_men(to_consider[1:], avail, memo)
            #Choose better branch
            if with_val > without_val:
                result = (with_val, with_team + (to_consider[0],))
            else:
                result = (without_val, without_team)
    memo[(to_consider, avail)] = result
    return result

In [5]:
shuffled_mini = tuple(random.sample(list(men_mini.index), len(men_mini.index)))
t0 = time.perf_counter()
val, taken = knapsack_men(shuffled_mini, (150, 1, 3, 25))
t1 = time.perf_counter()
print(val, taken)
print('{} seconds elapsed'.format(t1-t0))
print('Spent {} points on {} riders'.format(men_mini[men_mini.index.isin(taken)]['Price'].sum(), 
                                            len(men_mini[men_mini.index.isin(taken)])))
men_mini[men_mini.index.isin(taken)]

16504 (230, 150, 100, 0, 50, 190, 250, 200, 260, 110, 120, 140, 60, 240, 170, 180, 220, 210, 130, 10, 160, 30, 20, 90, 70)
47.2810489 seconds elapsed
Spent 147 points on 25 riders


,Rider Name,Price,Score 2019,24+,18+
0,Primož Roglič,28,4070,1,1
10,Mathieu van der Poel,8,1778,0,0
20,Mike Teunissen,2,1507,0,0
30,David Gaudu,2,1285,0,0
50,Philippe Gilbert,18,898,0,1
60,Jasper Philipsen,2,765,0,0
70,Guillaume Martin,6,695,0,0
90,Michael Valgren,14,620,0,0
100,Mads Pedersen,8,525,0,0
110,Pierre Latour,10,475,0,0


In [6]:
def knapsack_men_simple(to_consider, avail, memo={}):
    # to_consider is a tuple of rider indices
    # avail is simply an int of points remaining to spend
    # we're also going to query a global 'men' DataFrame variable, bite me
    
    if (to_consider, avail) in memo:
        result = memo[(to_consider, avail)]
    elif to_consider == () or avail == 0:
        result = (0, ())
    else:
        next_rider = men.loc[to_consider[0]]
        if next_rider['Price'] > avail:
            #Explore right branch only
            result = knapsack_men_simple(to_consider[1:], avail, memo)
        else:
            #Explore left branch
            with_val, with_team = knapsack_men_simple(to_consider[1:], avail - next_rider['Price'], memo)
            with_val += next_rider['Score 2019']
            #Explore right branch
            without_val, without_team = knapsack_men_simple(to_consider[1:], avail, memo)
            #Choose better branch
            if with_val > without_val:
                result = (with_val, with_team + (to_consider[0],))
            else:
                result = (without_val, without_team)
    memo[(to_consider, avail)] = result
    return result

In [7]:
shuffled_mini = tuple(random.sample(list(men_mini.index), len(men_mini.index)))
t0 = time.perf_counter()
val, taken = knapsack_men_simple(shuffled_mini, 150)
t1 = time.perf_counter()
print(val, taken)
print('{} seconds elapsed'.format(t1-t0))
print('Spent {} points on {} riders'.format(men_mini[men_mini.index.isin(taken)]['Price'].sum(), 
                                            len(men_mini[men_mini.index.isin(taken)])))
men_mini[men_mini.index.isin(taken)]

17439 (400, 160, 170, 230, 60, 390, 430, 140, 200, 330, 460, 240, 10, 190, 210, 100, 50, 310, 380, 110, 350, 360, 370, 0, 290, 120, 260, 450, 20, 340, 90, 250, 30, 280, 220, 320, 440, 420, 70, 470, 150)
2.0846666999999997 seconds elapsed
Spent 150 points on 41 riders


,Rider Name,Price,Score 2019,24+,18+
0,Primož Roglič,28,4070,1,1
10,Mathieu van der Poel,8,1778,0,0
20,Mike Teunissen,2,1507,0,0
30,David Gaudu,2,1285,0,0
50,Philippe Gilbert,18,898,0,1
60,Jasper Philipsen,2,765,0,0
70,Guillaume Martin,6,695,0,0
90,Michael Valgren,14,620,0,0
100,Mads Pedersen,8,525,0,0
110,Pierre Latour,10,475,0,0


In [8]:
men_mini.to_csv('men_mini.csv')